### Imports and functions

In [3]:
#basic imports
import pandas as pd
import numpy as np
from math import sqrt
from datetime import timedelta
import pickle

#data viz imports
import plotly.graph_objs as go
#use this format for working locally 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#model imports
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import fbprophet

#set numbers formatting 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
def region_df(region):
    
    '''
    Create a new dataframe from the main traffic_df, sectioning out the regions
    
    :region: the region (as a number) to create into own data frame

    Returns new dataframe with informations only pertaining to region 
    '''
    
    #section out region
    region_df= traffic_df[traffic_df.REGION_ID == region]
    
    #resets index
    region_df.reset_index(inplace=True, drop=True)
    
    #drop dups. This should have been corrected earlier, this is just a fail safe
    region_df.drop_duplicates(inplace= True)
    
    #remove data that does not pertain to local roads (over 40mph)
    #and observations where no cars are tracked (0mph)
    region_df = region_df[(region_df.SPEED < 40) & (region_df.SPEED != 0)]
    
    #return new data frame
    return region_df

def train_test_ts(df, holdout = .1):
    
    '''
    Given a time series dataframe, split into train and test
    
    :df: time series dataframe
    :holdout: The percent size for the test data

    Returns the train and test dataframes
    '''
    
    train_size = int(len(df) * (1-holdout))
    train, test = df[0:train_size], df[train_size:]
    
    return train, test

In [5]:
def Aug_Dicky_Fuller(column,stationary_count):
    '''
    Given a column of data going to be used for time series modeling,
    performs the Augmented Dickey Fuller stationarity test. 
    If n0 is rejected the data is stationary
    
    :column: the column of data, as a series or array
    :stationary_count: an int that used to track number of stationarity passed

    Returns stationary_count, increased by 1 if the data passed
    '''
    
    #null: non stationary data
    #alternative: stationary
    #reject null if p =< .05
    
    
    #p value
    #reject null, meaning data is stationary
    if adfuller(column)[1] <= .05:
        stationary_count += 1
        return stationary_count

    #accept null, meaning data has a trend
    else:
        return stationary_count 
    
def KPSS(column, stationary_count):
    '''
    Given a column of data going to be used for time series modeling,
    performs the Kwiatkowski–Phillips–Schmidt–Shin (KPSS) stationarity test. 
    If n0 is accepted the data is stationary
    
    :column: the column of data, as a series or array
    :stationary_count: an int that used to track number of stationarity passed

    Returns stationary_count, increased by 1 if the data passed
    '''
    
    #Null: stationary data
    #alternative: trend
    
    #reject null, meaning data has trend
    if kpss(column)[1] <= .05:
        return stationary_count 
    
    #accept null, meaning data does not have a trend
    else:
        stationary_count += 1
        return stationary_count

def mean_std_check(column, stationary_count):
    '''
    Given a column of data going to be used for time series modeling,
    split data in half and check the mean and standard deviation of each half.
    
    This method was obtained through some blog posts, it is not as robust as
    the previous methods
    
    :column: the column of data, as a series or array
    :stationary_count: an int that used to track number of stationarity passed

    Returns stationary_count, increased by 1 if the data passed
    '''

    #split data into two parts, check mean and standard deviation
    first_half_mean = round(column[:(len(column)//2)].mean())
    second_half_mean = round(column[(len(column)//2):].mean())

    first_half_std = round(column[:(len(column)//2)].std())
    second_half_std = round(column[(len(column)//2):].std())

    #check if the mean is constant over time
    if first_half_mean == second_half_mean:
        stationary_count += 1
        return stationary_count

    #else if the stand deviation is constant throughout the observations,
    #and the means are within a standard deviation, then it is stationary
    elif first_half_std == second_half_std:
        if abs(first_half_mean - second_half_mean) <= first_half_std:
            stationary_count += 1
            return stationary_count
        else:
            return stationary_count
    else:
        return stationary_count

def check_stationarity(column):    
    '''
    Given a column of data going to be used for time series modeling,
    performs 3 stationarity tests to check trend over time. 
    Returns True if 2 or more tests show stationarity in the data
    
    :column: the column of data, as a series or array

    Returns True if data passes at least two stationary test, else returns False
    '''

    
    #an in variable that will be fed to each tests, increasing by one for every test it passes
    stationary_count = 0

    #perform 3 tests to check for stationary, take the majority vote
    #each tests returns the stationary_count, increasing it by one if the data passes the test
    if mean_std_check(column,KPSS(column, Aug_Dicky_Fuller(column,stationary_count))) >= 2:
        return True
    else:
        return False
    
def forcast_error_metrics(dataframe,forecast):    
    '''
    Given dataframe of actual values and forecasted values, 
    returns a number of error metrics
    
    :dataframe: the dataframe with the real values
    :forecast: a series with the forecasted values

    Returns a number of error metrics based off predictions
    '''
    
    #timeseries database that you fit on
    #forecast is your prediction
    
    #the length of the dataframe
    df_len = len(dataframe)
    #pull the actual values from the dataframe
    y = dataframe.loc[:df_len, 'y'].values
    #pull the predicted values from data frame, removing the last value
    yhat = forecast.loc[:len(y)-1, 'yhat'].values
    
    #Forecast Error (or Residual Forecast Error)
    #actual minutes predicted
    for_err = y - yhat

    #Mean Forecast Error (or Forecast Bias) (or MAD)
    #the mean of the forecast error (on average, how off are you)
    mean_for_error = np.mean(for_err)
    #print('Mean Forcast Error (Forecast Bias): ',mean_for_error)
    
    #Mean Absolute Percent Error
    #what percent off are you usually?
    MAPE = np.mean(abs(y-yhat)/abs(y)) *100
    #print('\nMean Absolute Percent Error (MAPE): ',MAPE)
    
    #Mean Absolute Error(MAE)
    MAE = np.mean(abs(for_err))
    #print('\nMean Absolute Error(MAE): ',MAE)
    
    #Mean Squared Error(MSE)
    MSE = np.mean(for_err**2)
    #print('\nMean Squared Error(MSE): ',MSE)

    
    #Root Mean Squared Error(RMSE)
    RMSE = sqrt(MSE)
    #print('\nRoot Mean Squared Error(RMSE): ',RMSE)

    return mean_for_error,MAPE, RMSE
    

In [2]:
def time_series_cv(ts_df, splits = [.5, .8]):
    '''
    Given dataframe to train on, and cross validation splits,
    return error metrics of cross validations
    
    :dataframe: the data to train on
    :splits: a list of decimals to split data on

    Returns a number of error metrics based off predictions
    '''
    #cross validation
    
    error_metrics_list = []

    for split in splits:
        #split data
        train, validate = train_test_ts(ts_df, split)

        #fit on training data
        #this takes a bit
        ts_prophet = fbprophet.Prophet()
        ts_prophet.fit(train)

        #forcast on train and validate
        future = pd.DataFrame(pd.concat((train.ds, validate.ds), axis = 0))
        forcast = ts_prophet.predict(future)

        #measure error metrics for each cv
        error_metrics_list.append(forcast_error_metrics(ts_df, forcast))
        
    mean_forcast_bias = np.mean([item[0] for item in error_metrics_list])
    mean_MAPE = np.mean([item[1] for item in error_metrics_list])
    mean_RMSE = np.mean([item[2] for item in error_metrics_list])
    
    return mean_forcast_bias, mean_MAPE, mean_RMSE

In [6]:
def fb_prophet_funct(df, cv_on = False, splits = [.5, .8], hours_predict = 12):
    '''
    Given dataframe to train on, and cross validation splits,
    and future forecast, create model, cv (if turned on), and forecasted
    
    :dataframe: the data to train on
    :cv_on: should cross validations be included, autoset to off to save time
    :splits: a list of decimals to split data on
    :hours_predict: an int, the number of hours in the future to predict

    model, forecast, error, and cross validation results if set to True
    '''
    
    #restructure dataframes to be used for fb prophet
    ts = df[['TIME','SPEED']]
    ts = ts.rename(columns={'TIME': 'ds', 'SPEED': 'y'})
    
    #check stationarity, this part should take up to 30 seconds 
    if check_stationarity(ts.y) == True:
        
        if cv_on == True:
            cv_mean_forcast_bias, cv_mean_MAPE, cv_mean_RMSE = time_series_cv(ts)

        #final model
        ts_prophet = fbprophet.Prophet()
        ts_prophet.fit(ts)
        
        new_times = pd.DataFrame([ts.ds.max()+(i*timedelta(minutes = 10)) for i in range(2,(10*hours_predict)+1)],columns=['ds'])
        #new_times_df = pd.DataFrame(new_times_list,columns=['ds'])
        future = pd.DataFrame(pd.concat([ts.ds,new_times.ds], axis = 0))
        
        forecast = ts_prophet.predict(future)
        
        mean_forcast_bias, mean_MAPE, mean_RMSE = forcast_error_metrics(ts, forecast)
        
        if cv_on == True:
            return ts_prophet, future, forecast, mean_forcast_bias, mean_MAPE, mean_RMSE,\
                    cv_mean_forcast_bias, cv_mean_MAPE, cv_mean_RMSE
        else:
            return ts_prophet, future, forecast, mean_forcast_bias, mean_MAPE, mean_RMSE
                   
    else:
        print('Data is not stationary')

### Load data

In [8]:
traffic_df = pd.read_pickle('traffic_df.pkl')

traffic_df.head()

,BUS COUNT,NUMBER OF READS,REGION_ID,SPEED,TIME
0,12,215,5,25.230,2017-01-01 00:10:26
1,12,230,6,24.550,2017-01-01 00:10:26
2,10,150,1,28.640,2017-01-01 00:10:26
3,6,85,2,29.180,2017-01-01 00:10:26
4,20,288,3,27.950,2017-01-01 00:10:26


In [7]:
#look through each unique region id
for region in traffic_df.REGION_ID.unique():
        
        #create a dataframe with each unique region
        globals()['region%s'%(region)] = region_df(region)
        
        #split data into train and test for modeling purposes
        globals()['region%s_train'%(region)], globals()['region%s_test'%(region)] =\
        train_test_ts(globals()['region%s'%(region)])       

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Modeling

In [ ]:
for i in range(1,30):
    
    #create a model for each traffic region
    ts_prophet, future, forecast, mean_forcast_bias, mean_MAPE, mean_RMSE =\
    fb_prophet_funct(globals()['region%s'%(i)])
    
    #rename model, metrics, and forecast
    model_name = 'region%s_model'%(i)
    metric_name = 'region%s_stats'%(i)
    forecast_name = 'region%s_forecast'%(i)
    
    #pickle model, metrics, forecast
    with open(model_name, "wb") as f:
        pickle.dump(ts_prophet, f)
    
    with open(metric_name, "wb") as f:
        pickle.dump([mean_forcast_bias, mean_MAPE, mean_RMSE],f)
    
    with open(forecast_name, "wb") as f:
        pickle.dump(forecast,f)
    
    print(i)